# 🗺️ Plans 101

## 🏁 Let's kick this off with the right packages

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.1.0"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.1.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Planners.OpenAI, 1.1.0-preview"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.1.0-alpha"
#r "nuget: Microsoft.Extensions.Logging.Console, 8.0.0"

## 🔥 Fire up the kernel

⚠️ Note that if you're going to use the function-calling capabilities of the kernel, you'll need a function-calling compatible model. Please refer to [this chart](https://platform.openai.com/docs/guides/function-calling) on OpenAI's site. That's why in the example below I've inserted `gpt-4-1106-preview` into the slot because I tend to use function-calling a lot. But if you don't have access to that model on OpenAI, as of late January 2024 these models are possible as well:

* gpt-4
* gpt-4-1106-preview
* gpt-4-0613
* gpt-3.5-turbo
* gpt-3.5-turbo-1106
* gpt-3.5-turbo-0613

In [ ]:
#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Planning.Handlebars;
using Microsoft.Extensions.Logging;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI) {
    kernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey)
        .Build();
} else {
    kernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion("gpt-4-1106-preview", apiKey, orgId)
        .Build();
}

## ⌚️ Let's add a native C# plugin for use by the planner

In [ ]:
using System.ComponentModel;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Plugins.Core;

public class TimeInformationPlugin
{
    [KernelFunction]
    [Description("Retrieves the current time in UTC.")]
    public string GetCurrentUtcTime() => DateTime.UtcNow.ToString("R");
}

kernel.ImportPluginFromType<TimeInformationPlugin>();

## 📋 Let's keep track of all the plugins

In [ ]:
static void PrintAllPluginFunctions(Kernel kernel)
{
    var functions = kernel.Plugins.GetFunctionsMetadata();

    Console.WriteLine("****** Registered 🔌 Plugins and 📦 Functions ******");

    foreach (KernelFunctionMetadata func in functions)
    {
        PrintPluginFunction(func);
    }
}

static void PrintPluginFunction(KernelFunctionMetadata func)
{
    Console.WriteLine($"🔌 {func.PluginName}");
    Console.WriteLine($"   📦 /{func.Name}: {func.Description}");

    if (func.Parameters.Count > 0)
    {
        Console.WriteLine("      📥 Params:");
        foreach (var p in func.Parameters)
        {
            Console.WriteLine($"       • {p.Name}: {p.Description} (default: '{p.DefaultValue}')");
        }
    }
}

PrintAllPluginFunctions(kernel);

## 🗺️🚲 Generate a Plan from an ask

In [ ]:
#pragma warning disable SKEXP0060

 var planner = new HandlebarsPlanner();

var ask = @"Provide the current time and the name of the first president of the United States.";

var newPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("The proposed plan in Handlebars format:\n");
Console.WriteLine(newPlan);

## 🗺️🚲💨 Let's run the plan!

Note that the final output is json but the planner strips the extra JSON syntax.

In [ ]:
#pragma warning disable SKEXP0060

var newPlanResult = await newPlan.InvokeAsync(kernel, new KernelArguments());

newPlanResult


### 🗳️ JSON is always a good flavor, so let's have that result instead

In [ ]:
var kk = Utils.KeyValuePairsStringToJson(newPlanResult);

kk

## 🗺️🧊 You can also take an AI-generated Plan and edit it yourself

In [ ]:
#pragma warning disable SKEXP0060

string generatedPlanIsEditable =
"""
{{!-- Step 1: Retrieve the current UTC time --}}
{{set "currentTime" (TimeInformationPlugin-GetCurrentUtcTime)}}

{{!-- Step 2: Set the name of the first president of the United States --}}
{{set "firstName" "Jane"}}
{{set "lastName" "Washington"}}

{{!-- Step 3: Output the current time and the name of the first president --}}
{{json (concat "Current UTC Time: " currentTime ", First President: " firstName " " lastName)}}
""";

HandlebarsPlan editedPlan = new HandlebarsPlan(generatedPlanIsEditable);

var editedPlanResult = await editedPlan.InvokeAsync(kernel, new KernelArguments());

editedPlanResult


## 🗺️ 💾 And yes you can store that plan away for reuse

There isn't a standard way to store and reuse plans, but this is an example of how you could do it in concept.

### 🏁 YAML's a convenient format to use

In [ ]:
#r "nuget: YamlDotNet, 13.7.1"

### ℹ The basic parameters you would want to store

In [ ]:
using System.IO;
using YamlDotNet.Serialization;
using YamlDotNet.Serialization.NamingConventions;

public class Plany
{
    public string Name { get; set; }
    public string Plan { get; set; }
    public List<string> Plugins {get; set; }
    public string Description { get; set; }
}


### 🍱 The trick would be how you would maintain your Plugins, but you get the gist of it here

In [ ]:
List<string> planNames = ["TodayFirstPresident"];
List<Plany> allPlans = new List<Plany>();

foreach(var plan in planNames)
{
    var yaml = File.ReadAllText($"Plans/{plan}.yaml");
    var deserializer = new DeserializerBuilder()
        .WithNamingConvention(CamelCaseNamingConvention.Instance)
        .Build();

    var p = deserializer.Deserialize<Plany>(yaml);
    Console.WriteLine($"Name: {p.Name}\nPlan:\n```\n{p.Plan}```\nPlugins: {string.Join(", ", p.Plugins)}\nDescription: {p.Description}");

    allPlans.Add(p);
}

### 🔌 🚚 Let's make a simple PluginLoader to let you run this YAML format

There's not a single way to do this yet, but that shouldn't stop us from imagining the future and just running with it. For now ...

In [ ]:
using System;
using System.Collections.Generic;
using Microsoft.SemanticKernel;

// This was defined up above in a previous cell, AND it was already registered
//
// public class TimeInformationPlugin
// {
//     [KernelFunction]
//     [Description("Retrieves the current time in UTC.")]
//     public string GetCurrentUtcTime() => DateTime.UtcNow.ToString("R");
// }

public class RandomMaedaInformationPlugin
{
    [KernelFunction]
    [Description("Tells you something maeda randomly might say.")]
    public string GetCurrentUtcTime() => $"{DateTime.UtcNow.ToString("R")} says Maeda";
}

public class PluginLoader
{
    private readonly Kernel kernel;
    private readonly Dictionary<string, Action> pluginLoadActions;

    public PluginLoader(Kernel kernel)
    {
        this.kernel = kernel ?? throw new ArgumentNullException(nameof(kernel));
        pluginLoadActions = new Dictionary<string, Action>
        {
            { "TimeInformationPlugin", () => Import<TimeInformationPlugin>() },
            { "RandomMaedaInformationPlugin", () => Import<RandomMaedaInformationPlugin>() }
            // Add other plugins here...
        };
    }

    private void Import<T>() where T : new()
    {
        var pluginName = typeof(T).Name;
        if (!IsPluginLoaded(kernel, pluginName))
        {
            kernel.ImportPluginFromType<T>();
        }
        else
        {
            Console.WriteLine($" >> 🔌 '{pluginName}' is already loaded");
        }
    }

    public void ImportPlugin(string pluginName)
    {
        if (pluginLoadActions.TryGetValue(pluginName, out var action))
        {
            action();
        }
        else
        {
            throw new InvalidOperationException(" >> 🔌 Plugin not found");
        }
    }

    private static bool IsPluginLoaded(Kernel kernel, string pluginName)
    {
        var functions = kernel.Plugins.GetFunctionsMetadata();

        foreach (KernelFunctionMetadata func in functions)
        {
            if (func.PluginName.Equals(pluginName, StringComparison.OrdinalIgnoreCase))
            {
                return true; 
            }
        }

        return false; 
    }
}


### 🚚 With this little loader, we can now run Plans with ... Plugins :+).

In [ ]:
#pragma warning disable SKEXP0060

var pluginLoader = new PluginLoader(kernel);

foreach (var plan in allPlans)
{
    Console.WriteLine($"---\n🗺️ Plan: {plan.Name}");

    foreach (var pluginName in plan.Plugins)
    {
        Console.WriteLine($"🔌 Plugin: {pluginName}");
        pluginLoader.ImportPlugin(pluginName);
    }
    HandlebarsPlan thisPlan = new HandlebarsPlan(generatedPlanIsEditable);

    var thisPlanResult = await thisPlan.InvokeAsync(kernel, new KernelArguments());

    Console.WriteLine($"\n📤 Plan result:\n{thisPlanResult}\n---\n");
}

# 🗺️🧠 There's also a realtime planner called the FunctionCallingStepwisePlanner 

This planner is different from the handlebarsplanner in that it doesn't generate a plan ahead of time, and simply progresses towards its goal.

## 🔥 We first get a kernel ready

In [ ]:
#!import Plugins/EmailPlugin.cs

using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Planning;

Kernel kernel = Kernel.CreateBuilder()
    .AddOpenAIChatCompletion(
        apiKey: apiKey,
        modelId: "gpt-3.5-turbo-1106")
    .Build();

## 🔌 We then provide it a math SK Core plugin, and an inline one to simulate emailing

In [ ]:
#pragma warning disable SKEXP0050

kernel.ImportPluginFromType<MathPlugin>();
public class EmailSimPlugin
{
    [KernelFunction, Description("Given an e-mail and message body, send an email")]
    public string SendEmail(
        [Description("The body of the email message to send.")] string input,
        [Description("The email address to send email to.")] string email_address) {

            string result = $"Sent email to: {email_address}. Body: {input}";
            Console.WriteLine($" 🔌 EmailSimPlugin>> {result}");
            return result;
    }

    [KernelFunction, Description("Given a name, find email address")]
    public string GetEmailAddress(
        [Description("The name of the person whose email address needs to be found.")] string input)
    {
        string result = input switch
        {
            "Jane" => "janedoe4321@example.com",
            "Paul" => "paulsmith5678@example.com",
            "Mary" => "maryjones8765@example.com",
            _ => "johndoe1234@example.com",
        };

        Console.WriteLine($" 🔌 EmailSimPlugin>> Getting email address {result}");
        return result;
    }
}
kernel.ImportPluginFromType<EmailSimPlugin>();

PrintAllPluginFunctions(kernel);

## 🏃 Let's see it run

In [ ]:
#pragma warning disable SKEXP0050
#pragma warning disable SKEXP0060
#pragma warning disable SKEXP0061

string[] questions = {
            "Write a limerick, translate it to Spanish, and send it to Jane",
            "Mail the current time to Paul",
            "What is 387 minus 22? Email the solution to John and Mary.",
        };

var config = new FunctionCallingStepwisePlannerConfig
{
    MaxIterations = 15,
    MaxTokens = 4000,
};
var planner = new FunctionCallingStepwisePlanner(config);
int currentQuestion = 0;

foreach (var question in questions)
{
    Console.WriteLine($"🪜 Question {currentQuestion++}\nQ: {question}");
    FunctionCallingStepwisePlannerResult result = await planner.ExecuteAsync(kernel, question);
    Console.WriteLine($"A: {result.FinalAnswer}");

    // You can uncomment the line below to see the planner's process for completing the request.
    Console.WriteLine(Utils.WordWrap($"Chat history:\n{System.Text.Json.JsonSerializer.Serialize(result.ChatHistory)}", 200));
}

# 🧠 Last but not least, let's look at vanilla OpenAI-style function calling

You can disregard the Planners and simply use the OAI capability of calling functions, but still using Plugins.

## 🔥 Fire up a kernel

In [ ]:
Kernel kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI) {
    kernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey)
        .Build();
} else {
    kernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion("gpt-4-1106-preview", apiKey, orgId)
        .Build();
}

## 🔌 We make a simple plugin with C# code inline

In [ ]:
// Add a plugin with some helper functions we want to allow the model to utilize.
kernel.ImportPluginFromFunctions("HelperFunctions", new[]
{
    kernel.CreateFunctionFromMethod(() => DateTime.UtcNow.ToString("R"), "GetCurrentUtcTime", "Retrieves the current time in UTC."),
    kernel.CreateFunctionFromMethod((string cityName) =>
        cityName switch
        {
            "Boston" => "61 and rainy",
            "London" => "55 and cloudy",
            "Miami" => "80 and sunny",
            "Paris" => "60 and rainy",
            "Tokyo" => "50 and sunny",
            "Sydney" => "75 and sunny",
            "Tel Aviv" => "80 and sunny",
            _ => "31 and snowing",
        }, "Get_Weather_For_City", "Gets the current weather for the specified city"),
});

PrintAllPluginFunctions(kernel);

## 🏃 We can then run a prompt that auto-calls the functions available in registered plugins

⚠️ Note that if you're going to use the function-calling capabilities of the kernel, you'll need a function-calling compatible model. Please refer to [this chart](https://platform.openai.com/docs/guides/function-calling) on OpenAI's site. Make sure your kernel is using a model that supports function calling.

* gpt-4
* gpt-4-1106-preview
* gpt-4-0613
* gpt-3.5-turbo
* gpt-3.5-turbo-1106
* gpt-3.5-turbo-0613

In [ ]:
OpenAIPromptExecutionSettings settings = new() { ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions };
Console.WriteLine(await kernel.InvokePromptAsync("Given the current time of day and weather, what is the likely color of the sky in Boston?", new(settings)));

## 🏃💨 This is the same example as above, but with streaming too

In [ ]:
OpenAIPromptExecutionSettings settings = new() { ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions };
await foreach (var update in kernel.InvokePromptStreamingAsync("Given the current time of day and weather, what is the likely color of the sky in Boston?", new(settings)))
{
    Console.Write(update);
}

## 🔩 This version is for folks who like to do things manually. Functions are not auto-called.

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text.Json;
using System.Threading.Tasks;
using Azure.AI.OpenAI;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var chat = kernel.GetRequiredService<IChatCompletionService>();
var chatHistory = new ChatHistory();

OpenAIPromptExecutionSettings settings = new() { ToolCallBehavior = ToolCallBehavior.EnableKernelFunctions };
chatHistory.AddUserMessage("Given the current time of day and weather, what is the likely color of the sky in Boston?");
while (true)
{
    var result = (OpenAIChatMessageContent)await chat.GetChatMessageContentAsync(chatHistory, settings, kernel);

    if (result.Content is not null)
    {
        Console.Write(result.Content);
    }

    List<ChatCompletionsFunctionToolCall> toolCalls = result.ToolCalls.OfType<ChatCompletionsFunctionToolCall>().ToList();
    if (toolCalls.Count == 0)
    {
        break;
    }

    chatHistory.Add(result);
    foreach (var toolCall in toolCalls)
    {
        string content = kernel.Plugins.TryGetFunctionAndArguments(toolCall, out KernelFunction? function, out KernelArguments? arguments) ?
            JsonSerializer.Serialize((await function.InvokeAsync(kernel, arguments)).GetValue<object>()) :
            "Unable to find function. Please try again!";

        if (function != null)
        {
            Console.WriteLine($"  >> 🔌 {toolCall.Name.ToString()}: /{function.Name}");
        }

        Console.WriteLine($"       Result: {content}");

        chatHistory.Add(new ChatMessageContent(
            AuthorRole.Tool,
            content,
            metadata: new Dictionary<string, object?>(1) { { OpenAIChatMessageContent.ToolIdProperty, toolCall.Id } }));
    }
}